# Adicionando funções externas a API da OpenAI

Um grande salto de possibilidades de utilizações únicas da LLMs ocorrreu quando a OpenAI lançou o function calling. Essa ferramenta permite adicionarmos manualmente funções externas ao modelo que ele, dependendo da situação, poderá utilizar para obter novas informações ou atuar em diversos escopos. Vamos fazer uma breve revisão de como utilizamos funções externas na api da OpenAI, este assunto é explorado mais afundo no curso de Explorando a API da OpenAI. Na próxima aula, mostraremos como o framework langChain facilita a utilização das funções externas.

## Importações iniciais

## Criando função que será adicionada ao modelo

Utilizaremos uma função simples que simula uma api de tempo, que retorna a temperatura de um determinado local. Lembrando que modelos de llm são treinados com dados históricos, portanto, não possuem informações atuais. A única forma de eles entenderem o que está ocorrendo neste instante é passando informações pra eles através de prompts ou de funções externas.

'{"local": "Porto Alegre", "temperatura": "25", "unidade": "celsius"}'

## Criando descrição da função

Através dessa descrição o modelo entenderá o que a função faz e como ela pode ser utilizada

## Chamando o modelo com a nova ferramenta

Para chamar o modelo com a ferramenta criada, basta passar o argumento tools com uma lista de ferramentas.

ChatCompletion(id='chatcmpl-AsYOTubFNo4ttwzepR8eJEg4rTwvV', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_AIUxhBvCa31ctX3zjhs2P0Ji', function=Function(arguments='{"local":"Porto Alegre"}', name='obter_temperatura_atual'), type='function')]))], created=1737565285, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=87, total_tokens=110, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### Analisando a resposta

Podemos perceber que o conteúdo da resposta veio vazio, pois para a pergunta "Qual é a temperatura em Porto Alegre?" ele necessitará chamar a função antes.

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_AIUxhBvCa31ctX3zjhs2P0Ji', function=Function(arguments='{"local":"Porto Alegre"}', name='obter_temperatura_atual'), type='function')])

[ChatCompletionMessageToolCall(id='call_AIUxhBvCa31ctX3zjhs2P0Ji', function=Function(arguments='{"local":"Porto Alegre"}', name='obter_temperatura_atual'), type='function')]

obter_temperatura_atual
{"local":"Porto Alegre"}


### Adicionando resultado da função as mensagens

'{"local": "Porto Alegre", "temperatura": "25", "unidade": "celsius"}'

### Chamando novamente o modelo

[{'role': 'user', 'content': 'Qual é temperatura em Porto Alegre agora?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_AIUxhBvCa31ctX3zjhs2P0Ji', function=Function(arguments='{"local":"Porto Alegre"}', name='obter_temperatura_atual'), type='function')])]

[{'role': 'user', 'content': 'Qual é temperatura em Porto Alegre agora?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_AIUxhBvCa31ctX3zjhs2P0Ji', function=Function(arguments='{"local":"Porto Alegre"}', name='obter_temperatura_atual'), type='function')]),
 {'tool_call_id': 'call_AIUxhBvCa31ctX3zjhs2P0Ji',
  'role': 'tool',
  'name': 'obter_temperatura_atual',
  'content': '{"local": "Porto Alegre", "temperatura": "25", "unidade": "celsius"}'}]

ChatCompletion(id='chatcmpl-AsYOb5tZReoUBd3bOI2uolm17MVXH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='A temperatura em Porto Alegre agora é de 25°C.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1737565293, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=147, total_tokens=163, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

'A temperatura em Porto Alegre agora é de 25°C.'

## Explorando diferentes perguntas e o parâmetro tool_choice

Através do parâmetro tool_choice é possível forçar o modelo a sempre utilizar uma tool. Vamos ver como ele se comporta para diferentes perguntas modificando o parâmetro.

### Parâmetro "auto"

Assim o modelo define automaticamente se é necessária a utilização de uma função ou não

Conteúdo: None
Tools: [ChatCompletionMessageToolCall(id='call_hIB0t5IodL40cARvTUw3LDrT', function=Function(arguments='{"local":"Porto Alegre","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')]


Conteúdo: Olá! Como posso te ajudar hoje?
Tools: None


### Parâmetro "none"

Com o parâmetro "none", o modelo não vai utilizar funções.

Conteúdo: Vou verificar a temperatura atual em Porto Alegre para você. Apenas um momento, por favor.
Tools: None


Conteúdo: Olá! Como posso te ajudar hoje?
Tools: None


### Parâmetro "function"

Podemos fazer o modelo rodar obrigatoriamente a função, passando dentro de um dicionário a função que o modelo deve rodar.

Conteúdo: None
Tools: [ChatCompletionMessageToolCall(id='call_yZBuro6t06vP6pxAbpskf4HJ', function=Function(arguments='{"local":"Porto Alegre"}', name='obter_temperatura_atual'), type='function')]


Conteúdo: None
Tools: [ChatCompletionMessageToolCall(id='call_gGOsnHojNAg9seWLHG2qSCmy', function=Function(arguments='{"local":"São Paulo","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')]
